In [6]:
from datasets import load_from_disk
import torch
from torchvision import transforms
import numpy as np  # 新增numpy导入

# 确保train_data已加载
train_dir = rf'/hy-tmp/data/sample_0.2/train_dataset'
train_data = load_from_disk(train_dir)

In [8]:
train_data_100=train_data.select(range(100))

In [11]:
print(train_data_100)

Dataset({
    features: ['image', 'spectrum', 'params', 'z'],
    num_rows: 100
})


In [18]:
mean=train_data_100['image'].mean(dim=[0, 2, 3])
std=train_data_100['image'].std(dim=[0, 2, 3])
print(mean)
print(std)

tensor([0.0028, 0.0045, 0.0073, 0.0110, 0.0157])
tensor([0.0558, 0.0886, 0.1444, 0.2138, 0.3089])


In [19]:
# 新增全局像素统计
all_pixels = []
for i in range(len(train_data_100)):
    img = train_data[i]['image'].numpy()
    all_pixels.extend(img.flatten())

global_min = np.min(all_pixels)  # 使用显式np引用
global_max = np.max(all_pixels)  # 使用显式np引用
print(global_min, global_max)

-1.0234375 40.875


In [20]:
# 修改归一化函数为全局统计
def normalize_global(tensor):
    """使用全局像素范围进行归一化"""
    return (tensor - global_min) / (global_max - global_min + 1e-8)

In [23]:
image=train_data_100['image']
print(image.shape)

torch.Size([100, 5, 64, 64])


In [26]:
mean=train_data_100['image'].mean(dim=[0, 2, 3],keepdims=True)
std=train_data_100['image'].std(dim=[0, 2, 3],keepdims=True)

In [27]:
print(f"{image.min()}-{image.max()}")
image_mm=normalize_global(image)
print(f"{image_mm.min()}-{image_mm.max()}")

mean=image_mm.mean(dim=[0, 2, 3],keepdims=True)
std=image_mm.std(dim=[0, 2, 3],keepdims=True)
image_mm_norm=(image_mm-mean)/std
print(f"{image_mm_norm.min()}-{image_mm_norm.max()}")

-1.0234375-40.875
0.0-1.0
-4.78042459487915-163.7924041748047


In [31]:
x=np.log1p(image)
print(x.shape)

torch.Size([100, 5, 64, 64])


/tmp/ipykernel_648/450651239.py:1: RuntimeWarning: invalid value encountered in log1p
  x=np.log1p(image)
/tmp/ipykernel_648/450651239.py:1: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  x=np.log1p(image)


In [28]:
print(mean.shape)

torch.Size([1, 5, 1, 1])


In [21]:
# 创建transforms.Compose
transform = transforms.Compose([
    transforms.Lambda(lambda x: normalize_global(x)),
    transforms.Normalize(mean=mean.tolist(), std=std.tolist())
])

# 创建PairDataset实例
from dataset_util.PairDataset import PairDataset
pair_dataset = PairDataset(train_data_100, transform=transform)

In [2]:
train_dir=rf'/hy-tmp/data/sample_0.2/train_dataset'

In [3]:
train_data=load_from_disk(train_dir)

In [35]:
print(train_data)

Dataset({
    features: ['image', 'spectrum', 'params', 'z'],
    num_rows: 57797
})


In [40]:
image_all=train_data['image'].numpy()
print(image_all.shape)

(57797, 5, 64, 64)


In [54]:
print(image_all.shape)

(57797, 5, 64, 64)


In [ ]:
print(f"{image_all.min()}-{image_all.max()}")

In [41]:
min_val=-8.609375
max_val= 2032.0

In [ ]:
image_all

In [42]:
image_all_mm=(image_all-min_val)/(max_val-min_val)

In [55]:
print(f"{image_all_mm.min()}-{image_all_mm.max()}")

0.0-1.0


In [49]:
print(type(image_all_mm))
image_all_tensor=torch.from_numpy(image_all_mm)
print(type(image_all_tensor))

<class 'numpy.ndarray'>
<class 'torch.Tensor'>


In [47]:
image_all_mm.shape

(57797, 5, 64, 64)

In [50]:
mean=image_all_tensor.mean(dim=[0, 2, 3])
std=image_all_tensor.std(dim=[0, 2, 3])

In [52]:
nprint(mean)
print(std)

tensor([0.0042, 0.0042, 0.0042, 0.0042, 0.0042])
tensor([0.0002, 0.0001, 0.0002, 0.0002, 0.0006])


SSLMetaArch(
  (dino_loss): DINOLoss()
  (koleo_loss): KoLeoLoss(
    (pdist): PairwiseDistance()
  )
  (ibot_patch_loss): iBOTPatchLoss()
  (student): ModuleDict(
    (backbone): DinoVisionTransformer(
      (patch_embed): PatchEmbed(
        (proj): Conv2d(5, 1024, kernel_size=(8, 8), stride=(8, 8))
        (norm): Identity()
      )
      (blocks): ModuleList(
        (0): BlockChunk(
          (0-5): 6 x NestedTensorBlock(
            (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
            (attn): MemEffAttention(
              (qkv): Linear(in_features=1024, out_features=3072, bias=True)
              (attn_drop): Dropout(p=0.0, inplace=False)
              (proj): Linear(in_features=1024, out_features=1024, bias=True)
              (proj_drop): Dropout(p=0.0, inplace=False)
            )
            (ls1): LayerScale()
            (drop_path1): DropPath()
            (norm2): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
            (mlp): Mlp(
              (fc1): Linear(in_features=1024, out_features=4096, bias=True)
              (act): GELU(approximate='none')
              (fc2): Linear(in_features=4096, out_features=1024, bias=True)
              (drop): Dropout(p=0.0, inplace=False)
            )
            (ls2): LayerScale()
            (drop_path2): DropPath()
          )
        )
        (1): BlockChunk(
          (0-5): 6 x Identity()
          (6-11): 6 x NestedTensorBlock(
            (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
            (attn): MemEffAttention(
              (qkv): Linear(in_features=1024, out_features=3072, bias=True)
              (attn_drop): Dropout(p=0.0, inplace=False)
              (proj): Linear(in_features=1024, out_features=1024, bias=True)
              (proj_drop): Dropout(p=0.0, inplace=False)
            )
            (ls1): LayerScale()
            (drop_path1): DropPath()
            (norm2): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
            (mlp): Mlp(
              (fc1): Linear(in_features=1024, out_features=4096, bias=True)
              (act): GELU(approximate='none')
              (fc2): Linear(in_features=4096, out_features=1024, bias=True)
              (drop): Dropout(p=0.0, inplace=False)
            )
            (ls2): LayerScale()
            (drop_path2): DropPath()
          )
        )
        (2): BlockChunk(
          (0-11): 12 x Identity()
          (12-17): 6 x NestedTensorBlock(
            (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
            (attn): MemEffAttention(
              (qkv): Linear(in_features=1024, out_features=3072, bias=True)
              (attn_drop): Dropout(p=0.0, inplace=False)
              (proj): Linear(in_features=1024, out_features=1024, bias=True)
              (proj_drop): Dropout(p=0.0, inplace=False)
            )
            (ls1): LayerScale()
            (drop_path1): DropPath()
            (norm2): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
            (mlp): Mlp(
              (fc1): Linear(in_features=1024, out_features=4096, bias=True)
              (act): GELU(approximate='none')
              (fc2): Linear(in_features=4096, out_features=1024, bias=True)
              (drop): Dropout(p=0.0, inplace=False)
            )
            (ls2): LayerScale()
            (drop_path2): DropPath()
          )
        )
        (3): BlockChunk(
          (0-17): 18 x Identity()
          (18-23): 6 x NestedTensorBlock(
            (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
            (attn): MemEffAttention(
              (qkv): Linear(in_features=1024, out_features=3072, bias=True)
              (attn_drop): Dropout(p=0.0, inplace=False)
              (proj): Linear(in_features=1024, out_features=1024, bias=True)
              (proj_drop): Dropout(p=0.0, inplace=False)
            )
            (ls1): LayerScale()
            (drop_path1): DropPath()
            (norm2): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
            (mlp): Mlp(
              (fc1): Linear(in_features=1024, out_features=4096, bias=True)
              (act): GELU(approximate='none')
              (fc2): Linear(in_features=4096, out_features=1024, bias=True)
              (drop): Dropout(p=0.0, inplace=False)
            )
            (ls2): LayerScale()
            (drop_path2): DropPath()
          )
        )
      )
      (norm): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
      (head): Identity()
    )
    (dino_head): DINOHead(
      (mlp): Sequential(
        (0): Linear(in_features=1024, out_features=2048, bias=True)
        (1): GELU(approximate='none')
        (2): Linear(in_features=2048, out_features=2048, bias=True)
        (3): GELU(approximate='none')
        (4): Linear(in_features=2048, out_features=256, bias=True)
      )
      (last_layer): Linear(in_features=256, out_features=65536, bias=False)
    )
  )
  (teacher): ModuleDict(
    (backbone): DinoVisionTransformer(
      (patch_embed): PatchEmbed(
        (proj): Conv2d(5, 1024, kernel_size=(8, 8), stride=(8, 8))
        (norm): Identity()
      )
      (blocks): ModuleList(
        (0): BlockChunk(
          (0-5): 6 x NestedTensorBlock(
            (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
            (attn): MemEffAttention(
              (qkv): Linear(in_features=1024, out_features=3072, bias=True)
              (attn_drop): Dropout(p=0.0, inplace=False)
              (proj): Linear(in_features=1024, out_features=1024, bias=True)
              (proj_drop): Dropout(p=0.0, inplace=False)
            )
            (ls1): LayerScale()
            (drop_path1): Identity()
            (norm2): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
            (mlp): Mlp(
              (fc1): Linear(in_features=1024, out_features=4096, bias=True)
              (act): GELU(approximate='none')
              (fc2): Linear(in_features=4096, out_features=1024, bias=True)
              (drop): Dropout(p=0.0, inplace=False)
            )
            (ls2): LayerScale()
            (drop_path2): Identity()
          )
        )
        (1): BlockChunk(
          (0-5): 6 x Identity()
          (6-11): 6 x NestedTensorBlock(
            (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
            (attn): MemEffAttention(
              (qkv): Linear(in_features=1024, out_features=3072, bias=True)
              (attn_drop): Dropout(p=0.0, inplace=False)
              (proj): Linear(in_features=1024, out_features=1024, bias=True)
              (proj_drop): Dropout(p=0.0, inplace=False)
            )
            (ls1): LayerScale()
            (drop_path1): Identity()
            (norm2): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
            (mlp): Mlp(
              (fc1): Linear(in_features=1024, out_features=4096, bias=True)
              (act): GELU(approximate='none')
              (fc2): Linear(in_features=4096, out_features=1024, bias=True)
              (drop): Dropout(p=0.0, inplace=False)
            )
            (ls2): LayerScale()
            (drop_path2): Identity()
          )
        )
        (2): BlockChunk(
          (0-11): 12 x Identity()
          (12-17): 6 x NestedTensorBlock(
            (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
            (attn): MemEffAttention(
              (qkv): Linear(in_features=1024, out_features=3072, bias=True)
              (attn_drop): Dropout(p=0.0, inplace=False)
              (proj): Linear(in_features=1024, out_features=1024, bias=True)
              (proj_drop): Dropout(p=0.0, inplace=False)
            )
            (ls1): LayerScale()
            (drop_path1): Identity()
            (norm2): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
            (mlp): Mlp(
              (fc1): Linear(in_features=1024, out_features=4096, bias=True)
              (act): GELU(approximate='none')
              (fc2): Linear(in_features=4096, out_features=1024, bias=True)
              (drop): Dropout(p=0.0, inplace=False)
            )
            (ls2): LayerScale()
            (drop_path2): Identity()
          )
        )
        (3): BlockChunk(
          (0-17): 18 x Identity()
          (18-23): 6 x NestedTensorBlock(
            (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
            (attn): MemEffAttention(
              (qkv): Linear(in_features=1024, out_features=3072, bias=True)
              (attn_drop): Dropout(p=0.0, inplace=False)
              (proj): Linear(in_features=1024, out_features=1024, bias=True)
              (proj_drop): Dropout(p=0.0, inplace=False)
            )
            (ls1): LayerScale()
            (drop_path1): Identity()
            (norm2): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
            (mlp): Mlp(
              (fc1): Linear(in_features=1024, out_features=4096, bias=True)
              (act): GELU(approximate='none')
              (fc2): Linear(in_features=4096, out_features=1024, bias=True)
              (drop): Dropout(p=0.0, inplace=False)
            )
            (ls2): LayerScale()
            (drop_path2): Identity()
          )
        )
      )
      (norm): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
      (head): Identity()
    )
    (dino_head): DINOHead(
      (mlp): Sequential(
        (0): Linear(in_features=1024, out_features=2048, bias=True)
        (1): GELU(approximate='none')
        (2): Linear(in_features=2048, out_features=2048, bias=True)
        (3): GELU(approximate='none')
        (4): Linear(in_features=2048, out_features=256, bias=True)
      )
      (last_layer): Linear(in_features=256, out_features=65536, bias=False)
    )
  )
)